In [3]:
%pylab widget
%load_ext autoreload

%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from varian_models import *

### Now what would fastCat think of this?

#### We can loop through this

This runs off of a modified xpecgen that returns only the doses and 

In [5]:
import sys
import os
import tigre
from scipy.signal import fftconvolve

import fastcat.fastcat as fc

dose = 0

phan_map_hard =  ['air','water','water','water',
             'water','water','water','water']


phan_map_g4 = ['air','water','G4_LUNG_ICRP',"G4_BONE_COMPACT_ICRU","G4_BONE_CORTICAL_ICRP","G4_ADIPOSE_TISSUE_ICRP","G4_BRAIN_ICRP","G4_B-100_BONE"] 

spectrum_files = sorted(['120_kVp','Al_spectrum_25', 'C_spectrum_6', 'Al_spectrum_6', 'C_spectrum_25', 'W_spectrum_6', 'W_spectrum_25'])
# spectrum_files = sorted(['Al_spectrum_25', 'C_spectrum_6', 'Al_spectrum_6', 'C_spectrum_25', 'W_spectrum_6', 'W_spectrum_25'])

dets = ['CuGOS-784-micrometer']

phan_maps = [phan_map_hard]#,phan_map_soft]

angles = np.linspace(0,2*np.pi,2)

res = []

original_energies_keV = np.array([30, 40, 50 ,60, 70, 80 ,90 ,100 ,300 ,500 ,700, 900, 1000 ,2000 ,4000 ,6000])


angles = np.linspace(0,2*pi,2)

geo = 'catphan_low_contrast_512'

phantom = fc.Catphan_515()

for phan_map in phan_maps:
    for spectrum_file in spectrum_files[:]:
        for det in dets:
            
            if spectrum_file == '120_kVp':
                s = fc.calculate_spectrum(120, 12, 3, 50,monitor=None)
                s.attenuate(0.1,fc.get_mu(z=13))
            else:
                s = fc.Spectrum()
                s.load(spectrum_file)
        
            kernel = fc.Kernel(s, det)
            phantom.phan_map = phan_map_hard
            res.append(phantom.return_projs(kernel,s,angles,
                                       nphoton=2e7,
                                      mgy=dose,return_dose=True))

(19, 41, 41) 19
bowtie is off False
really doing it!
ratio is 1.0 number of photons 0.07104375195840185
(19, 41, 41) 19
bowtie is off False
really doing it!
ratio is 1.0 number of photons 0.04845504309805526
(19, 41, 41) 19
bowtie is off False
really doing it!
ratio is 1.0 number of photons 0.03402763640736402
(19, 41, 41) 19
bowtie is off False
really doing it!
ratio is 1.0 number of photons 0.04026543917603216
(19, 41, 41) 19
bowtie is off False
really doing it!
ratio is 1.0 number of photons 0.03243039447096263
(19, 41, 41) 19
bowtie is off False
really doing it!
ratio is 1.0 number of photons 0.01015508406165716
(19, 41, 41) 19
bowtie is off False
really doing it!
ratio is 1.0 number of photons 0.0053004634067323865


In [6]:
kv_catphan = Catphan_404_kV()
# phan_map_hard =  ['air','water','water','water',
#              'water','water','water','water',
#              'water','water','water','water',
#              'water','water','water','water',
#              'water','water','water','water',
#              'water','water','water','water']
# kv_catphan.phan_map = phan_map_hard
# kv_catphan.geomet.sVoxel = np.array((160, 160, 160)) 
# kv_catphan.geomet.dVoxel = kv_catphan.geomet.sVoxel/kv_catphan.geomet.nVoxel
kv_catphan.get_proj(angles,fudge=0.,ASG=True,return_dose=True,nphoton=2e7)#,bowtie=False)
res.append(kv_catphan.res)

# phan_map_hard =  ['air','water','water','water',
#              'water','water','water','water',
#              'water','water','water','water',
#              'water','water','water','water',
#              'water','water','water','water',
#              'water','water','water','water']
kv_catphan = Catphan_404_6x()
# kv_catphan.phan_map = phan_map_hard
# kv_catphan.geomet.sVoxel = np.array((160, 160, 160)) 
# kv_catphan.geomet.dVoxel = kv_catphan.geomet.sVoxel/kv_catphan.geomet.nVoxel
kv_catphan.get_proj(angles,fudge=0.,return_dose=True,nphoton=2e7)#,bowtie=False)
res.append(kv_catphan.res)

(19, 19, 19) 19
bowtie is off False
Using the bowtie filter True
really doing it!
bowtie scatter
ratio is 1.0 number of photons 0.6742965492673298
(19, 41, 41) 19
bowtie is off False
Using the bowtie filter True
really doing it!
ratio is 1.0 number of photons 0.02636808212587352


In [20]:
plt.figure()
plt.plot(np.array([res[ii][-1] for ii in [-2,-1]]),'b*')
doses_add = np.array([
    5.806760627891094e-10/1000000,
    9.877950013503778e-09/1000000
])
# doses_add = np.array([
#     4.964694e-10/1000000,
#     9.32785013503778e-09/1000000
# ])
doses_add = np.array([
    3.602318698147385e-10/1000000,
    7.9612928871679e-09/1000000
])
# the mc dose is 1.46 times smaller than fastcat

plt.plot(4.53*doses_add/(1.6021766e-16 * 1000),'r*')
# plt.plot(4.53*doses_add,'r*')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [52]:
plt.figure()
plt.plot(np.array([res[ii][-1] for ii in [-2,-1]])*(1.6021766e-16 * 1000)*2e8,'b*')
doses_add = np.array([
    5.806760627891094e-10/1000000,
    9.877950013503778e-09/1000000
])
# doses_add = np.array([
#     4.964694e-10/1000000,
#     9.32785013503778e-09/1000000
# ])
doses_add = np.array([
    3.602318698147385e-10/1000000,
    7.9612928871679e-09/1000000
])
# the mc dose is 1.46 times smaller than fastcat

plt.plot(4.53*(doses_add)*2e8+1e-7,'r*')
# plt.plot(4.53*doses_add,'r*')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
doses = np.array([
6.033596460814806e-08,
1.613708349168519e-07,
1.313530049174005e-07,
6.200432949299284e-07,
3.304057543118688e-07,
2.999242099122057e-07,
3.515523019359534e-07,
])
# Doses for the real beams, kv , 6X, both with the filters.
doses_add = np.array([
    2.401448484512514e-10/1800000,
    2.368887308534795e-08/10000000
])
names = np.array([
'Dose_120_kVp',
'Dose_Al_spectrum_25',
'Dose_C_spectrum_25',
'Dose_W_spectrum_6',
'Dose_Al_spectrum_6', 
'Dose_C_spectrum_6',
'Dose_W_spectrum_25'])

indeces = np.argsort(names)

names = names[indeces]
doses = doses[indeces]

indeces = np.argsort(doses)

names = names[indeces]
doses = doses[indeces]

In [9]:
doses/20000000/(1.6021766e-16 * 1000)
doses_fc2 = np.array([res[ii][-1] for ii in range(len(res))])
# doses_fc2 = np.array(doses_fc)
doses_fc2 = doses_fc2[indeces]

In [10]:
kv_catphan.s.x.shape

(41,)

In [14]:
plt.figure()
plt.plot(kv_catphan.s.x,kv_catphan.s.y)
# print(kv_catphan.s.y/np.sum(kv_catphan.s.y))
print(kv_catphan.s.x)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[1.00000000e+00 2.00000000e+00 4.45398893e+01 7.42331489e+01
 7.46336360e+01 8.98521467e+01 9.03384525e+01 1.05757207e+02
 1.06958668e+02 1.52270925e+02 1.53672630e+02 1.70321452e+02
 1.86569787e+02 2.02589272e+02 2.18723182e+02 2.93299605e+02
 3.37467614e+02 3.81263741e+02 4.10299058e+02 4.54009366e+02
 4.98291800e+02 4.97662463e+02 5.11650906e+02 6.13231604e+02
 7.29515901e+02 8.75121576e+02 1.04990560e+03 1.26885763e+03
 1.44407075e+03 1.75115855e+03 2.02915383e+03 2.30734935e+03
 2.67350900e+03 3.03989751e+03 3.62629647e+03 4.21286708e+03
 4.75555574e+03 5.22486943e+03 5.65015815e+03 5.85540780e+03
 5.97277914e+03]


In [15]:
plt.figure()
# s = fc.calculate_spectrum(100, 14, 3, 100,monitor=None)
xx = np.array(s.get_points()).T
plt.plot(xx[:,0],xx[:,1])
np.save('data/energies.npy',xx[:,0])
np.save('data/spec_100.npy',xx[:,1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
import pandas as pd

In [18]:
plt.figure()#figsize=[8,4],dpi=300)

plt.subplot(121)
dat = pd.DataFrame(d_norm/20000000/(1.6021766e-16 * 1000),index=shorts)
# plt.plot(doses_fc2,d_norm/20000000/(1.6021766e-16 * 1000),'kx')
pp = np.polyfit(doses_fc2,d_norm/20000000/(1.6021766e-16 * 1000),1)
xx = np.linspace(0.040,0.2,100)
yy = np.polyval(pp,xx)
# plt.plot(xx,yy,':',color='cornflowerblue')
# plt.ylabel('fastCAT')
# plt.xlabel('Topas')
slope, intercept, r_value, p_value, std_err = sp.stats.linregress(doses_fc2,d_norm/20000000/(1.6021766e-16 * 1000),)
# plt.legend(['Simulations',f'linear fit: y = {np.round(pp[0],3)}x + {np.round(pp[1],3)}, \n $r^2$= {np.round(r_value,4)}'])
# pp = np.array([0.88759883, 0.01035186])
plt.plot(doses_fc2,'*',color='cornflowerblue')
plt.plot(dat,'k+')
# plt.plot(doses_fc/sum(doses_fc),'x')
plt.xticks(rotation=75)
plt.ylabel('MeV/photon')
plt.legend(['fastCAT','Topas'])
plt.grid()
plt.title('Dose Comparison')


plt.subplot(122)

nphot = 360*1e9

dat2 = [489/1000000*nphot,27.86/1000000*nphot,61.1,39.9]

plt.semilogy(dat2,'k*')
plt.grid('on')
plt.ylabel('log compute time [core s]')
plt.xticks([0,1,2,3],labels=['Topas MC*','fastEPID*','fastCAT 1024$^2$x10','fastCAT 512$^2$x10'],rotation= 45)
plt.yticks([60,60*60,60*60*24,60*60*24*30,60*60*24*365],['minute','hour','day','month','year'])
plt.title('Compute Time')

plt.tight_layout()
plt.savefig('/home/xcite/fastcat_paper/figures/misc.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'd_norm' is not defined

In [19]:
import pandas as pd
import scipy as sp
# d_norm = doses#/sum(doses)
# doses_fc2 = np.array(doses_fc)
# doses_fc2 = doses_fc2[indeces]
shorts = ['120 kVp', 'C 2.5 MV', 'Al 2.5 MV', 'C 6 MV',
       'Al 6 MV', 'W 2.5 MV', 'W 6 MV']
fig = plt.figure()#figsize=[8,4],dpi=300)
ax1 = fig.add_subplot(121)
scats = np.load('../paper_2/curves_all.npy')
real_spec = np.load('../data/w_spec_profile.npy')
real_spec_CT = np.load('/home/xcite/scintillators/Catphan_dose/batch_submission_W/analysis/Readout_0000.phsp.npy')
real_spec_kV = np.load('/home/xcite/scintillators/topas_sim_cugos/kV_averaged_slice.npy')
dist = np.linspace(-256*0.0784 - 0.0392,256*0.0784 - 0.0392, 512)
plt.plot(dist,scats[0],linewidth = 1.0,color = 'cornflowerblue')
plt.plot(dist,scats[1],linewidth = 1.0,color = 'darkorange')
plt.plot(dist,scats[2],linewidth = 1.0,color = 'gray')
plt.plot(dist,real_spec,'k-',linewidth = 0.4)
plt.plot(dist,np.mean(real_spec_CT,0),'k-.',linewidth = 0.4,markersize=0.4)
dist2 = dist*1.04
plt.plot(dist2,real_spec_kV,'k--',linewidth = 0.6)
plt.plot(dist,scats[2],linewidth = 1.0,color = 'gray')
# plt.grid('on')
plt.xlim([-20,20])
plt.ylim([0,1000])
fig.legend(['fastCAT 6 MV Al', 'fastCAT 6 MV W',
            'fastCAT 120 kVp','Topas 6 MV Al', 'Topas 6 MV W', 'Topas 120 kVp'],
           loc='upper right', bbox_to_anchor=(0.6,0.97))
plt.xlabel('[cm]')
plt.ylabel('Counts')

axins = ax1.inset_axes([0.26, 0.45, 0.44, 0.54])
axins.plot(dist,scats[0],linewidth = 1.0,color = 'cornflowerblue')
axins.plot(dist,scats[1],linewidth = 1.0,color = 'darkorange')
axins.plot(dist,scats[2],linewidth = 1.0,color = 'gray')
axins.plot(dist,real_spec,'k-',linewidth = 0.4)
axins.plot(dist,np.mean(real_spec_CT,0),'k-.',linewidth = 0.4,markersize=0.4)
axins.plot(dist2,real_spec_kV,'k--',linewidth = 0.6)
axins.plot(dist,scats[2],linewidth = 1.0,color = 'gray')
x1, x2, y1, y2 = -8, 8, 10, 55
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticklabels('')
axins.set_yticklabels('')
ax1.indicate_inset_zoom(axins)

topas = np.load('/home/xcite/scintillators/Catphan_dose/batch_submission/projs_new_npys.npy')
fastCat = np.load('/home/xcite/fastcat-0.0.1/fastcat/data/projs/similar_noise8.npy')#fastCat_topas_equiv_8cm_W.npy')
ax = fig.add_subplot(122)#,box_aspect=1.5)
plt.fill_between(np.linspace(-20,20,512),np.mean(fastCat.T[:,:,0],1)/10-np.std(fastCat.T[:,:,0],1)/10, 
                 np.mean(fastCat.T[:,:,0],1)/10+np.std(fastCat.T[:,:,0],1)/10 ,alpha=0.3, facecolor='cornflowerblue')
plt.fill_between(np.linspace(-20,20,512),np.mean(topas.T[:,:64,0],1)-np.std(topas.T[:,:64,0],1),
                 np.mean(topas.T[:,:64,0],1)+np.std(topas.T[:,:64,0],1),alpha=0.3, facecolor='k')
plt.plot(np.linspace(-20,20,512),np.mean(fastCat.T[:,:,0],1)/10,lineWidth= .5,color='b')
plt.plot(np.linspace(-20,20,512),np.mean(topas.T[:,:64,0],1),lineWidth= .5,color='k')
plt.plot(np.linspace(-20,20,512),np.mean(topas.T[:,:64,0],1)+np.std(topas.T[:,:64,0],1),'k--',lineWidth= .5)
plt.plot(np.linspace(-20,20,512),np.mean(fastCat.T[:,:,0],1)/10+np.std(fastCat.T[:,:,0],1)/10,'--',lineWidth= .5,color='cornflowerblue',alpha=0.8)
plt.plot(np.linspace(-20,20,512),np.mean(topas.T[:,:64,0],1)-np.std(topas.T[:,:64,0],1),'k--',lineWidth= .5)
plt.plot(np.linspace(-20,20,512),np.mean(fastCat.T[:,:,0],1)/10-np.std(fastCat.T[:,:,0],1)/10,'--',lineWidth= .5,color='cornflowerblue',alpha=0.8)
plt.legend(['$\hat{\mu}$ fastCAT', '$\hat{\mu}$ Topas', '$\hat{\sigma}$ Topas', '$\hat{\sigma}$ fastCAT'],loc='upper right')
# inset axes....
axins = ax.inset_axes([0.3, 0.05, 0.4, 0.4])
axins.fill_between(np.linspace(-20,20,512),np.mean(fastCat.T[:,:,0],1)/10-np.std(fastCat.T[:,:,0],1)/10, 
                 np.mean(fastCat.T[:,:,0],1)/10+np.std(fastCat.T[:,:,0],1)/10 ,alpha=0.3, facecolor='cornflowerblue')
axins.fill_between(np.linspace(-20,20,512),np.mean(topas.T[:,:64,0],1)-np.std(topas.T[:,:64,0],1),
                 np.mean(topas.T[:,:64,0],1)+np.std(topas.T[:,:64,0],1),alpha=0.3, facecolor='k')
axins.plot(np.linspace(-20,20,512),np.mean(topas.T[:,:64,0],1)+np.std(topas.T[:,:64,0],1),'k--',lineWidth= .5)
axins.plot(np.linspace(-20,20,512),np.mean(fastCat.T[:,:,0],1)/10+np.std(fastCat.T[:,:,0],1)/10,'--',lineWidth= .5,color='cornflowerblue',alpha=0.8)
axins.plot(np.linspace(-20,20,512),np.mean(topas.T[:,:64,0],1)-np.std(topas.T[:,:64,0],1),'k--',lineWidth= .5)
axins.plot(np.linspace(-20,20,512),np.mean(fastCat.T[:,:,0],1)/10-np.std(fastCat.T[:,:,0],1)/10,'--',lineWidth= .5,color='cornflowerblue',alpha=0.8)
axins.plot(np.linspace(-20,20,512),np.mean(fastCat.T[:,:,0],1)/10,lineWidth= .5,color='b')
axins.plot(np.linspace(-20,20,512),np.mean(topas.T[:,:64,0],1),lineWidth= .5,color='k')

plt.xlim([-20,20])
plt.ylim([-0.5,1.5])
# plt.grid()
# sub region of the original image
x1, x2, y1, y2 = -5, 5, 0.8, 1.1
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticklabels('')
axins.set_yticklabels('')
plt.ylabel('$\mu x$')
plt.xlabel('[cm]')
plt.tight_layout()
ax.indicate_inset_zoom(axins)
# plt.subplot(223)
# # plt.plot(doses_fc2,d_norm/20000000/(1.6021766e-16 * 1000),'kx')
# pp = np.polyfit(doses_fc2,d_norm/20000000/(1.6021766e-16 * 1000),1)
# xx = np.linspace(0.040,0.2,100)
# yy = np.polyval(pp,xx)
# # plt.plot(xx,yy,':',color='cornflowerblue')
# # plt.ylabel('fastCAT')
# # plt.xlabel('Topas')
# slope, intercept, r_value, p_value, std_err = sp.stats.linregress(doses_fc2,d_norm/20000000/(1.6021766e-16 * 1000),)
# # plt.legend(['Simulations',f'linear fit: y = {np.round(pp[0],3)}x + {np.round(pp[1],3)}, \n $r^2$= {np.round(r_value,4)}'])
# plt.plot(doses_fc2*pp[0] + pp[1],'*',color='cornflowerblue')
# plt.plot(dat,'k+')
# # plt.plot(doses_fc/sum(doses_fc),'x')
# plt.xticks(rotation=75)
# plt.ylabel('MeV/photon')
# plt.legend(['fastCAT','Topas'])
plt.tight_layout()
# plt.savefig('/home/xcite/fastcat_paper/figures/one_slice_comparison_2panel.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …